In [ ]:
from scripts.exp.researcher.idea_pool import Idea, Idea_Pool

idea_pool = Idea_Pool(cache_path=r"scripts/exp/researcher/output_dir/idea_pool/test.json")

In [ ]:
import json
from rdagent.oai.llm_utils import APIBackend


system_prompt = """You are analyzing several ideas when running some kaggle competitions on data science.
User has found several possible good ideas and you are asked to identify what component those ideas are focusing.

Possible components are:
- Feature Engineering including anything related to data preprocessing like augmentation, cleaning, etc.
- Model related including anything related to model selection, model architecture, hyperparameter tuning, etc.
- Model ensemble
- Others

You should provide the component for each idea and only return the component in the following json schema:
{
    "idea_id": "component",
    "idea_id": "component",
    ...
}
component should be one of "feature", "model", "ensemble", "others"
"""


# for index, idea in enumerate(idea_pool.idea_pool[:3]):
for index in range(0, len(idea_pool.idea_pool), 10):
    user_prompt = ""
    part_idea_pool = idea_pool.idea_pool[index: index + 10]
    for inner_index, idea in enumerate(part_idea_pool):
        user_prompt += f"idea_{inner_index}:\nIdea: {idea.idea}\n Method: {idea.method}\n Context: {idea.context}\n"

    resp = json.loads(APIBackend().build_messages_and_create_chat_completion(user_prompt, system_prompt, json_mode=True))
    for k, v in resp.items():
        try:
            idea_pool.idea_pool[index + int(k.split("_")[-1])].target_component = v
        except:
            pass


In [13]:
import pickle 
pickle.dump(idea_pool, open("scripts/exp/researcher/output_dir/idea_pool/test.pkl", "wb"))